In [1]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install pymc3
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats, odr
import statsmodels.api as sm
import random

In [3]:
def print_confidence_interval(param, stdev, alpha=0.05):
    confidence_interval = np.array([param] * 2) + \
                        stats.norm.ppf(1-alpha/2) * np.multiply([-1, 1], [stdev] * 2)

    print('[{0:.3f}	{1:.3f}]'.format(alpha/2, 1-alpha/2))
    print(' {0:.3f}	{1:.3f}'.format(confidence_interval[0], confidence_interval[1]))

<p class="gap2">
<h1 style="font-weight: bold; color: #ed9041">Some Attention for Attenuation Bias<br>
</h1>

<h2 style="color: #667b83">And how we account for it in geo experiments</h2>


<p class="gap05">  </p>
<h3 style="color: #459db9">Ruben Mak <br>
Principal Data Scientist at Greenhouse</h3>
<img src="images/pydata_global_logo.png">
</p>

## Short introduction

<p><img src="images/wpp_greenhouse.png"></p>

<p><img src="images/bias.png"></p>

## Perspective on bias for this talk
* Computing something different than you intent to do

## Attenuation bias
* You want to compute: $\hat{\beta}$ in $y = \alpha + \beta x + \epsilon_y$ 
* $\epsilon_y \sim \mathcal{N}(0,\sigma_y)$

* But you are not observing the true $x$ but $\tilde{x} = x + \epsilon_x$
* $\epsilon_x \sim \mathcal{N}(0,\sigma_x)$

* So you are computing $\hat{\beta}$ in$y=\alpha+\beta(x+\epsilon_x)+\epsilon_y$

* Attenuation Bias = Measurement Error Bias = Regression Dilution

In [4]:
import statsmodels.api as sm

𝛼 = 0 
𝛽 = 2.5
𝜎𝑦 = 𝜎𝑥 = 1
size = 10000

true_𝑥 = np.random.normal(loc=5, scale=1, size=size)
𝑥 = true_𝑥 + np.random.normal(loc=0, scale=𝜎𝑥, size=size)
𝑦 = 𝛼 + 𝛽 * true_𝑥 + np.random.normal(loc=0, scale=𝜎𝑦, size=size)

X = sm.add_constant(𝑥)
model = sm.OLS(𝑦,X)
results = model.fit()

In [5]:
print_confidence_interval(results.params[1], np.sqrt(results.cov_params()[1,1]))
print('𝛽 = {}'.format(𝛽))
results.summary()

[0.025	0.975]
 1.226	1.283
𝛽 = 2.5


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.430
Method:                 Least Squares   F-statistic:                     7543.
Date:                Fri, 29 Oct 2021   Prob (F-statistic):               0.00
Time:                        17:59:00   Log-Likelihood:                -21271.
No. Observations:               10000   AIC:                         4.255e+04
Df Residuals:                    9998   BIC:                         4.256e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.2821      0.075     83.930      0.000       6.135       6.429
x1             1.2545      0.014     86.851      0.000       1.226       1.283
==============================================================================
Omnibus:                        3.707   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.157   Jarque-Bera (JB):                3.693
Skew:                          -0.037   Prob(JB):                        0.158
Kurtosis:                       3.058   Cond. No.                         19.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## When to be aware of potential attenuation bias?
* There might be noise in your observed $\tilde{x}$

* You care about what happens to $y$ when true $x$ changes

* You're doing inference, not plain prediction or classification

## Are you doing inference?
* Let me illustrate with a short story...

<p><img src="images/scenario_1.png"></p>

<p><img src="images/scenario_2.png"></p>

<p><img src="images/scenario_3.png"></p>

<p><img src="images/scenario_4.png"></p>

<p><img src="images/scenario_5.png"></p>

## Beyond linear models
* More general, you want to know what happens to $y$ when true $x$ is changed: $\delta_y = \mathbb{E}(y | x=v_2, z) - \mathbb{E}(y | x=v_1, z)$

* Relation to causal inference $\mathbb{P}(Y | X) \neq \mathbb{P}(Y | Do(X))$

## Reasons why I think attenuation bias is overlooked
* Focus on prediction / machine learning

* Academia are conservative in rejecting the $H_0$

## The direction of attenuation bias
* Attenuation bias is always towards zero: $E(|\hat{\beta}|) < |\beta|$

* Academia: if we find a significant effect without accounting for attenuation bias, the effect is also significant when accounting for it

* Academia: but if we incorrectly account for attenuation bias, we might be falsely rejecting $H_0$

* But in practice, underestimation can be just as costly as overestimation!

## Accounting for attenuation bias
* Deming Regression, Orthogonal (Distance) Regression (ODR), Total Least Squares (TLS): same family of solutions

* Error-in-Variables models, flexible Bayesian version using latent variables. Includes applications in deep learning ([May 2021](https://arxiv.org/abs/2105.09095)).

* In my experience, Error-in-Variables models can be quite sensitive to model specification. See [this notebook](https://github.com/rubenmak/pydata_global_2021_attenuation_bias/blob/master/Bayesian_EiV_is_biased.ipynb).

* Our proposed solution for binary treatment

In [6]:
from scipy import odr

𝛼 = 0 
𝛽 = 2.5
𝜎𝑦 = 𝜎𝑥 = 1
size = 10000

true_𝑥 = np.random.normal(loc=5, scale=1, size=size)
𝑥 = true_𝑥 + np.random.normal(loc=0, scale=𝜎𝑥, size=size)
𝑦 = 𝛼 + 𝛽 * true_𝑥 + np.random.normal(loc=0, scale=𝜎𝑦, size=size)

def f(B, x):
    return B[0] + B[1]*x

linear = odr.Model(f)
mydata = odr.RealData(𝑥, 𝑦)
myodr = odr.ODR(mydata, linear, beta0=[1., 2.])

results = myodr.run()
# results.pprint()

print_confidence_interval(results.beta[1], results.sd_beta[1])
print('𝛽 = {}'.format(𝛽))

[0.025	0.975]
 2.411	2.507
𝛽 = 2.5


## ODR / TLS assumptions
* Here, we assume $\sigma_x = \sigma_y$
* You need to at least know something about $\sigma_x$, $\sigma_y$ and/or $\dfrac{\sigma_x}{\sigma_y}$

In [7]:
𝛼 = 0 
𝛽 = 2.5
𝜎𝑦 = 0.1
𝜎𝑥 = 1
size = 10000

true_𝑥 = np.random.normal(loc=5, scale=1, size=size)
𝑥 = true_𝑥 + np.random.normal(loc=0, scale=𝜎𝑥, size=size)
y = 𝛼 + 𝛽 * true_𝑥 + np.random.normal(loc=0, scale=𝜎𝑦, size=size)

def f(B, x):
    return B[0] + B[1]*x

linear = odr.Model(f)
mydata = odr.RealData(𝑥, 𝑦)
myodr = odr.ODR(mydata, linear, beta0=[1., 2.])

results = myodr.run()
# results.pprint()

print_confidence_interval(results.beta[1], results.sd_beta[1])
print('𝛽 = {}'.format(𝛽))

[0.025	0.975]
 2.166	2.245
𝛽 = 2.5


In [8]:
𝛼 = 0 
𝛽 = 2.5
𝜎𝑦 = 0.1
𝜎𝑥 = 1
size = 10000

true_𝑥 = np.random.normal(loc=5, scale=1, size=size)
𝑥 = true_𝑥 + np.random.normal(loc=0, scale=𝜎𝑥, size=size)
y = 𝛼 + 𝛽 * true_𝑥 + np.random.normal(loc=0, scale=𝜎𝑦, size=size)

def f(B, x):
    return B[0] + B[1]*x

linear = odr.Model(f)
mydata = odr.RealData(𝑥, 𝑦, sx=𝜎𝑥, sy=𝜎𝑦)
myodr = odr.ODR(mydata, linear, beta0=[1., 2.])

results = myodr.run()
# results.pprint()

print_confidence_interval(results.beta[1], results.sd_beta[1])
print('𝛽 = {}'.format(𝛽))

[0.025	0.975]
 2.431	2.526
𝛽 = 2.5


## Geo Experiments
* Make an assignment of treatments to different geographical regions

* Alternative to A/B test (Randomized Controlled Trial)

* Reason 1: outcome cannot be measured in treated / non-treated groups, [Do mobile banner ads increase sales? Yes, in the offline channel](https://ink.library.smu.edu.sg/lkcsb_research/6243/)

* Reason 2: Impossible to measure non-treated outcome: Google Ads (Search)

<p><img src="images/paid_organic.png"></p>

## Geo Experiments

* Google paper 2011 [Measuring Ad Effectiveness Using Geo Experiments](https://research.google/pubs/pub38355/)
* Google paper mentioning orthogonal regression 2017 [Estimating Ad Effectiveness using Geo Experiments in a Time-Based Regression Framework](https://research.google/pubs/pub45950/)

<p><img src="images/geo_dma.png"></p>


In [9]:
df1 = pd.read_csv('data/geo_data_1.csv')
df1

,time,geo,treated,outcome
0,0,0,0,1000
1,0,1,0,1500
2,1,0,0,1200
3,1,1,1,2700


## Attenuation Bias in Geo Experiments

* Why do we have noise in $x$?

* We want to know the effect of treatment $T$ (true $x$) but we observe geo ($\tilde{x}$)

* Reason 1: Different technologies used for estimating location, some are noisy (store, mobile phone, website)

* Reason 2: People move between treated and non-treated geo

* Under some assumptions, we have data to estimate $\sigma_{x}$

## Proposed Solution
* Simple but powerful trick

* Only works for binary $x$

* Bernoulli distribution: $\sigma^2 = p(1-p)$

* Knowing $\sigma_x$ implies $p_x$

In [10]:
df2 = pd.read_csv('data/geo_data_2.csv')
df2

,time,geo,treated,outcome
0,0,0,0.00,1000
1,0,1,0.00,1500
2,1,0,0.15,1200
3,1,1,0.85,2700


## Intuition
* Geo's aren't fully treated / non-treated, but 'partly treated / non-treated'

## Generic example with binary treatment

In [11]:
import statsmodels.api as sm

𝛼 = 0 
𝛽 = 2.5
𝜎𝑦 = 1
𝑝𝑥 = 0.85
size = 10000
      
apply_noise = lambda x: 1 - x if random.uniform(0, 1) > 𝑝𝑥 else x
apply_noise = np.vectorize(apply_noise)
        
true_𝑥 = np.random.binomial(n=1, p=0.5, size=size)
𝑥 = apply_noise(true_𝑥)
𝑦 = 𝛼 + 𝛽 * true_𝑥 + np.random.normal(loc=0, scale=𝜎𝑦, size=size)

X = sm.add_constant(𝑥)
model = sm.OLS(𝑦,X)
results = model.fit()

print_confidence_interval(results.params[1], np.sqrt(results.cov_params()[1,1]))
print('𝛽 = {}'.format(𝛽))

[0.025	0.975]
 1.737	1.842
𝛽 = 2.5


In [12]:
import statsmodels.api as sm

𝛼 = 0 
𝛽 = 2.5
𝜎𝑦 = 1
𝑝𝑥 = 0.85
size = 10000
      
apply_noise = lambda x: 1 - x if random.uniform(0, 1) > 𝑝𝑥 else x
apply_noise = np.vectorize(apply_noise)
      
apply_p = lambda x: 𝑝𝑥 if x == 1 else 1 - 𝑝𝑥
apply_p = np.vectorize(apply_p)
        
true_𝑥 = np.random.binomial(n=1, p=0.5, size=size)
𝑥 = apply_noise(true_𝑥)
𝑝 = apply_p(𝑥)
𝑦 = 𝛼 + 𝛽 * true_𝑥 + np.random.normal(loc=0, scale=𝜎𝑦, size=size)

X = sm.add_constant(𝑝)
model = sm.OLS(𝑦,X)
results = model.fit()

print_confidence_interval(results.params[1], np.sqrt(results.cov_params()[1,1]))
print('𝛽 = {}'.format(𝛽))

[0.025	0.975]
 2.403	2.553
𝛽 = 2.5


In [13]:
𝛼 = 0 
𝛽 = 2.5
𝜎𝑦 = 1
𝑝𝑥 = 0.85
𝜎𝑥 = np.sqrt(𝑝𝑥 * (1 - 𝑝𝑥))
size = 10000
      
apply_noise = lambda x: 1 - x if random.uniform(0, 1) > 𝑝𝑥 else x
apply_noise = np.vectorize(apply_noise)
        
true_𝑥 = np.random.binomial(n=1, p=0.5, size=size)
𝑥 = apply_noise(true_𝑥)
𝑦 = 𝛼 + 𝛽 * true_𝑥 + np.random.normal(loc=0, scale=𝜎𝑦, size=size)

linear = odr.Model(f)
mydata = odr.RealData(𝑥, 𝑦, sx=𝜎𝑥, sy=𝜎𝑦)
myodr = odr.ODR(mydata, linear, beta0=[1., 2.])

results = myodr.run()
# results.pprint()

print_confidence_interval(results.beta[1], results.sd_beta[1])
print('𝛽 = {}'.format(𝛽))
print('𝜎𝑥 = {0:.3f}'.format(𝜎𝑥))

[0.025	0.975]
 3.539	3.694
𝛽 = 2.5
𝜎𝑥 = 0.357


## Applications
* We apply this trick for analysis of our geo experiments

* Application in double machine learning and other methods for causal inference

* When imputing missing data:
    * Andrew Gelman suggests adding noise to make imputed data more realistic (Data Analysis Using Regression and Multilevel/Hierarchical Models)
    * If you use the imputed variable for inference, you'll be introducing double attenuation bias!
    * For binary variables, impute $p_x$ instead

* [How Using Machine Learning Classification as a Variable in Regression Leads to Attenuation Bias and What to Do About It, October 2021](https://osf.io/preprints/socarxiv/453jk/)

## When to be aware of potential attenuation bias?
* There might be noise in your observed $\tilde{x}$

* You care about what happens to $y$ when true $x$ changes

* You're doing inference, not plain prediction or classification

## Thanks!
* Tobias Klein
* Koen Graat
* Ernst Osinga, Menno Zevenbergen, Mark van Zuijlen
* Google

## Questions?

In [14]:
!jupyter nbconvert pydata_global_2021_attenuation_bias.ipynb --to slides --post serve

[NbConvertApp] Converting notebook pydata_global_2021_attenuation_bias.ipynb to slides
[NbConvertApp] Writing 642937 bytes to pydata_global_2021_attenuation_bias.slides.html
[NbConvertApp] Redirecting reveal.js requests to https://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.5.0
Serving your slides at http://127.0.0.1:8000/pydata_global_2021_attenuation_bias.slides.html
Use Control-C to stop this server
^C

Interrupted
